In [5]:
from matplotlib import pyplot                                 #importing relevant libraries
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np
import math
import pandas as pd
from scipy import stats
import scipy

In [6]:
def newtons_method(llf, llfd,lmb, tolerance=0.00000001): #defining a function which will run the Newton-Raphson algorithm
    while True:
        lmb1 = lmb - llf(lmb)/llfd(lmb)
        t = abs(lmb1 - lmb)                             #difference between obtained value and starting point
        if t < tolerance:                      
            break
        lmb = lmb1                                      #algorithm stops if the difference is less than tolerance value
    return lmb

In [7]:
df=pd.read_csv("C:\Users\Rawal Rawal\Downloads\salesdata.csv")     #importing the dataset
df.index=pd.to_datetime(df['Date'])
series = df['Sales']
x =np.array(series)                                     #x gives us the time series values

In [8]:
                                                       #defining the log likelihood function,which has to be maximized
def llf(lmb):
    y =np.empty([len(x),],float)
    N=len(x)
    for i in x:
        if lmb == 0:                                   #the box-cox transformation is defined as log(x) if lambda=0
            i = float(i)
            x1 = math.log(i)
            y = np.append(y,[x1],axis=0)
        else:
            i = float(i)
            x1 = ((i**lmb)-1)/lmb                       #using the general formula given in Box,Cox(1972)
            y = np.append(y,[x1],axis=0)
    y1 = y[len(x):]
    y1_mean = np.mean(y1, axis=0)
    llf = (lmb - 1) * np.sum(np.log(x), axis=0)              
    llf -= N / 2.0 * np.log(np.sum((y1 - y1_mean)**2. / N)) 
    return llf

def llfd(lmb):                                         #taking first derivative of the function,which has to be equated to zero
        llfd=  (llf(lmb+0.000001) - llf(lmb))/0.000001
        return llfd
    
def llfdd(lmb):                                        #second derivative of log likelihood function
    llfdd=  (llfd(lmb+0.000001) - llfd(lmb))/0.000001
    return llfdd

lmbe = scipy.stats.boxcox_normmax(x)                #choosing a starting point for lambda,as the value generated by maximizing 
                                                    #the pearson correlation coefficient between normally distributed values and
                                                    #the given values
        
y= newtons_method(llfd,llfdd,lmb=lmbe)              #applying the function defined above

print (y)

0.875967727266318


In [9]:
s2,s3 = stats.boxcox_normmax(x,method = 'all')      #comapring with the values generated by maximization of
                                                    #pearson correlation coefficient and using brent optimization to maximize
                                                    #the loglikeihood function
s2,s3 

(0.9345902232871635, 0.8759682281367331)